In [ ]:
%pip install geopandas
%pip install rasterio
%pip install scipy

# Develop and Test GIS Functions

This notebook is for developing and testing the GIS functions implemented in `functions.py` using open-source libraries such as geopandas, rasterio, numpy, and scipy.

In [ ]:
# Import Required Libraries
import os
import sys
import unittest
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np
import scipy
import rasterio
from rasterio.mask import mask


In [ ]:
# Import functions from functions.py
cur_dir = os.path.abspath('.')
if cur_dir not in sys.path:
    sys.path.append(cur_dir)  # Ensure current directory is in path
sys.path

In [ ]:
def RPL_Select_analysis(input_shp, output_shp, conditions_expression):
    """
    Selects features from the input shapefile based on a given expression and saves them to an output shapefile.
    
    Parameters:
    - input_shp: Path to the input shapefile.
    - output_shp: Path to save the output shapefile.
    - conditions_expression: Expression to filter features. Example: "TA2025_V1_ == '001'".
    
    Returns:
    - None
    """
    gdf = gpd.read_file(input_shp)
    selected_gdf = gdf.query(conditions_expression)
    selected_gdf.to_file(output_shp)

def RPL_Clip_analysis(output, whole_area, boundary):
    """
    Clips a whole area shapefile to a specified boundary and saves the result.
    
    Parameters:
    - output: Path to save the clipped shapefile.
    - whole_area: Path to the whole area shapefile.
    - boundary: Path to the boundary shapefile.
    
    Returns:
    - None
    """
    whole_area_gdf = gpd.read_file(whole_area)
    boundary_gdf = gpd.read_file(boundary)
    
    clipped_gdf = gpd.clip(whole_area_gdf, boundary_gdf)
    clipped_gdf.to_file(output)

def RPL_Buffer_analysis(input, output, buffer_distance):
    """
    Creates a buffer around features in a shapefile and saves the result.
    Users can use `[axis.unit_name for axis in gdf.crs.axis_info]` to get the unit of a shapefile.
    
    Parameters:
    - input: Path to the input shapefile.
    - output: Path to save the buffered shapefile.
    - buffer_distance: Distance for the buffer (float, same unit as input shapefile).

    Returns:
    - None
    """
    gdf = gpd.read_file(input)
    buffered_gdf = gdf.buffer(distance=buffer_distance)
    buffered_gdf.to_file(output)

def RPL_Union_analysis(inputs, output):
    """
    Unions multiple shapefiles into a single shapefile, merging geometries and discarding attributes.
    
    Parameters:
    - inputs: List of paths to input shapefiles.
    - output: Path to save the unioned shapefile.
    
    Returns:
    - None
    """
    gds_list = [gpd.read_file(input).geometry for input in inputs]
    gs_all = gpd.GeoSeries(pd.concat(gds_list, ignore_index=True), crs=gds_list[0].crs)
    unioned =  gs_all.union_all()
    unioned.to_file(output)

def RPL_ExtractByMask(input_raster, mask_shapefile, output_raster):
    """
    Extract the cells of a raster that fall within the area defined by a shapefile mask, setting other cells to nodata.
    
    Parameters:
    - input_raster: Path to the input raster file.
    - mask_shapefile: Path to the shapefile defining the mask.
    
    Returns:
    - None
    """
    mask_gdf = gpd.read_file(mask_shapefile)
    mask_geoms = mask_gdf.geometry.values

    # Open the raster file
    with rasterio.open(input_raster) as src:
        # Mask the raster with the shapefile geometry
        out_image, out_transform = mask(src, mask_geoms, crop=True)

        # Copy the metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": src.nodata  # Ensure nodata is preserved
        })

    # Save the masked raster to file
    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)


In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from matplotlib import pyplot

annual_solar_radiation_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/lris-lenz-mean-annual-solar-radiation-GTiff/solar_2193.tif'
slope_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/lris-lenz-slope-GTiff/slop_2193.tif'
temperature_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/lris-lenz-mean-annual-temperature-GTiff/temperature_2193.tif'
district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/district.shp'
solar_in_district_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/solar-in-district.tif'
slope_in_district_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/slope-in-district.tif'
temperature_in_district_raster = '/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/temperature-in-district.tif'


RPL_ExtractByMask(annual_solar_radiation_raster, district_shp, solar_in_district_raster)
RPL_ExtractByMask(slope_raster, district_shp, slope_in_district_raster)
RPL_ExtractByMask(temperature_raster, district_shp, temperature_in_district_raster)


In [ ]:
src = rasterio.open(solar_in_district_raster)
pyplot.imshow(src.read(1))
pyplot.show()

In [ ]:
# union buffered lake and residential areas in a specific district
buffered_lake_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/buffered-lake-in-district.shp'
buffered_residential_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/buffered-residential-in-district.shp'
unioned_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/unioned-in-district.shp'
gdf1 = gpd.read_file(buffered_lake_in_district_shp)
gdf2 = gpd.read_file(buffered_residential_in_district_shp)
gs_all = gpd.GeoSeries(pd.concat([gdf1.geometry, gdf2.geometry], ignore_index=True), crs=gdf1.crs)

# gs_all.plot(figsize=(10, 10),facecolor='none', edgecolor='black')
unioned =  gpd.GeoSeries([gs_all.union_all()], crs=gs_all.crs)
unioned.plot(figsize=(10, 10), facecolor='none', edgecolor='red')
RPL_Union_analysis([buffered_lake_in_district_shp, buffered_residential_in_district_shp], unioned_in_district_shp)
gpd.read_file(unioned_in_district_shp).plot(figsize=(10, 10), facecolor='none', edgecolor='blue')

In [ ]:
# Create buffer for lake areas in a specific district
lake_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/lake-in-district.shp'
buffered_lake_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/buffered-lake-in-district.shp'
RPL_Buffer_analysis(
    input=lake_in_district_shp,
    output=buffered_lake_in_district_shp,
    buffer_distance=500
)
gpd.read_file(buffered_lake_in_district_shp).plot(figsize=(10, 10))

In [ ]:
# Create buffer for residential areas in a specific district
residential_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/residential-in-district.shp'
buffered_residential_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/buffered-residential-in-district.shp'
RPL_Buffer_analysis(
    input=residential_in_district_shp,
    output=buffered_residential_in_district_shp,
    buffer_distance=1000
)
gpd.read_file(buffered_residential_in_district_shp).plot(figsize=(10, 10))

In [ ]:
# clip the residential polygons to the district boundary
all_residential_shp = '/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/lds-nz-residential-area-polygons-topo-150k-SHP/nz-residential-area-polygons-topo-150k.shp'
district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/district.shp'
residential_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/residential-in-district.shp'
RPL_Clip_analysis(
    output=residential_in_district_shp,
    whole_area=all_residential_shp,
    boundary=district_shp
)

In [ ]:
gpd.read_file(residential_in_district_shp).plot(figsize=(10, 10))

In [ ]:
# clip the lake polygons to the district boundary
all_lake_shp = '/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/lds-nz-lake-polygons-topo-150k-SHP/nz-lake-polygons-topo-150k.shp'
district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/district.shp'
lake_in_district_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/lake-in-district.shp'
RPL_Clip_analysis(
    output=lake_in_district_shp,
    whole_area=all_lake_shp,
    boundary=district_shp
)

In [ ]:
gpd.read_file(lake_in_district_shp).plot(figsize=(10, 10))

In [ ]:
# extract a specific district (e.g., '002') from the territorial authority shapefile
input_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/test-data/statsnz-territorial-authority-2025-clipped-SHP/territorial-authority-2025-clipped.shp'
output_shp='/Users/landy/UoW/COMPX576/code/site-analyzer/output-data/district.shp'
RPL_Select_analysis(
    input_shp=input_shp,
    output_shp=output_shp,
    conditions_expression=f"TA2025_V1_ == '024'"
)
gpd.read_file(output_shp).plot(figsize=(10, 10))